### Loading Data

In [ ]:
file1 = open('/content/drive/MyDrive/Colab Notebooks/datasets/train.txt', 'r')
lines = file1.readlines()
lines[:5]

['i didnt feel humiliated;sadness\n',
 'i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake;sadness\n',
 'im grabbing a minute to post i feel greedy wrong;anger\n',
 'i am ever feeling nostalgic about the fireplace i will know that it is still on the property;love\n',
 'i am feeling grouchy;anger\n']

In [ ]:
import re
counter = 0
for line in lines:
  pattern = re.compile("\n")
  new_line = pattern.sub("", line)
  pattern_new = re.compile(";")
  lines[counter] = pattern_new.split(new_line)
  counter+=1
lines[:5]

[['i didnt feel humiliated', 'sadness'],
 ['i can go from feeling so hopeless to so damned hopeful just from being around someone who cares and is awake',
  'sadness'],
 ['im grabbing a minute to post i feel greedy wrong', 'anger'],
 ['i am ever feeling nostalgic about the fireplace i will know that it is still on the property',
  'love'],
 ['i am feeling grouchy', 'anger']]

In [ ]:
import pandas as pd
data = pd.DataFrame(lines, columns =["line", "emotion"])
data.head()

,line,emotion
0,i didnt feel humiliated,sadness
1,i can go from feeling so hopeless to so damned...,sadness
2,im grabbing a minute to post i feel greedy wrong,anger
3,i am ever feeling nostalgic about the fireplac...,love
4,i am feeling grouchy,anger


In [ ]:
data.groupby("emotion").count()

,line
emotion,
anger,2159
fear,1937
joy,5362
love,1304
sadness,4666
surprise,572


In [ ]:
anger = data.groupby("emotion").get_group('anger').sample(800) d3
fear = data.groupby("emotion").get_group('fear').sample(800) b5
joy = data.groupby("emotion").get_group('joy').sample(800) a4
love = data.groupby("emotion").get_group('love').sample(800) g4
sadness = data.groupby("emotion").get_group('sadness').sample(800) c7
surprise = data.groupby("emotion").get_group('surprise').sample(570) f7
data = pd.concat([anger, fear, joy, love, sadness, surprise]).sample(frac=1)
data.head()

,line,emotion
1503,i was feeling brave tonight so i decided to go...,joy
2341,i wasn t feeling insulted over its idiocy i fe...,anger
6750,i am feeling a little rejected by my sister,sadness
9745,im still feeling really shitty and undeserving...,sadness
6443,i feel vicious and sleepy,anger


### Preprocessing

In [ ]:
import nltk
import numpy as np

nltk.download('punkt')
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup

wordnet_lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
stopwords = set(w.rstrip() for w in open('/content/drive/MyDrive/Colab Notebooks/datasets/reviews/extras/stopwords.txt'))

In [ ]:
data = data.values
word_map = {}
counter = 0

def my_tokenizer(s):
  s = s.lower()
  tokens = nltk.tokenize.word_tokenize(s)
  tokens = [x for x in tokens if len(x) > 2]
  tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
  tokens = [t for t in tokens if t not in stopwords]
  return tokens

for n in range(0,data.shape[0]):
  words = my_tokenizer(data[n][0])
  data[n][0] = words
  for word in words:
    if word not in word_map:
      word_map[word] = counter
      counter+=1

In [ ]:
data[0]

array([list(['wa', 'feeling', 'brave', 'tonight', 'decided', 'attempt', 'vlog']),
       'joy'], dtype=object)

In [ ]:
def token_to_vector(tokens, label):
  x = np.zeros(len(word_map)+1)
  for t in tokens:
    i = word_map[t]
    x[i] +=1
  x = x / x.sum()
  x[-1] = label
  return x

data_train = np.zeros((data.shape[0], len(word_map)+1))
i = 0
for tokens,label in data:
  if label == "anger":
    l=0
  elif label == "fear":
    l=1
  elif label == "joy":
    l=2
  elif label == "love":
    l=3
  elif label == "sadness":
    l=4
  else:
    l=5
  pt = token_to_vector(tokens, l)
  data_train[i,:] = pt
  i+=1

In [ ]:
data_train[0]

array([0.14285714, 0.14285714, 0.14285714, ..., 0.        , 0.        ,
       2.        ])

### Model

In [ ]:
x = data_train[:,:-1]
y = data_train[:,-1]

In [ ]:
x.shape, y.shape

((4570, 6699), (4570,))

In [ ]:
x_train = x[:-650, :]
y_train = y[:-650, ]
x_test = x[-650:, :]
y_test = y[-650:, ]

In [ ]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((3920, 6699), (650, 6699), (3920,), (650,))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(500, n_jobs=-2, verbose = 2)
model.fit(x_train, y_train)

[Parallel(n_jobs=-2)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=-2)]: Done   1 out of   1 | elapsed:    0.2s remaining:    0.0s


building tree 1 of 500
building tree 2 of 500
building tree 3 of 500
building tree 4 of 500
building tree 5 of 500
building tree 6 of 500
building tree 7 of 500
building tree 8 of 500
building tree 9 of 500
building tree 10 of 500
building tree 11 of 500
building tree 12 of 500
building tree 13 of 500
building tree 14 of 500
building tree 15 of 500
building tree 16 of 500
building tree 17 of 500
building tree 18 of 500
building tree 19 of 500
building tree 20 of 500
building tree 21 of 500
building tree 22 of 500
building tree 23 of 500
building tree 24 of 500
building tree 25 of 500
building tree 26 of 500
building tree 27 of 500
building tree 28 of 500
building tree 29 of 500
building tree 30 of 500
building tree 31 of 500
building tree 32 of 500
building tree 33 of 500
building tree 34 of 500
building tree 35 of 500
building tree 36 of 500
building tree 37 of 500
building tree 38 of 500
building tree 39 of 500
building tree 40 of 500
building tree 41 of 500
building tree 42 of 500
b

[Parallel(n_jobs=-2)]: Done 500 out of 500 | elapsed:  1.5min finished


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-2, oob_score=False, random_state=None, verbose=2,
                       warm_start=False)

In [ ]:
y_pred = model.predict(x_test)
from sklearn.metrics import accuracy_score, classification_report
accuracy_score(y_test, y_pred)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.4s finished


0.8369230769230769

In [ ]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.80      0.89      0.84       123
         1.0       0.88      0.81      0.84       122
         2.0       0.79      0.73      0.76       120
         3.0       0.88      0.93      0.91       119
         4.0       0.84      0.77      0.81       102
         5.0       0.83      0.91      0.87        64

    accuracy                           0.84       650
   macro avg       0.84      0.84      0.84       650
weighted avg       0.84      0.84      0.84       650



### Testing 1 => Kate Chopin - The Story of an Hour

In [ ]:
test_text = "Knowing that Mrs Mallard was afflicted with a heart trouble, great care was taken to break to her as gently as possible the news of her husband's death. It was her sister Josephine who told her, in broken sentences; veiled hints that revealed in half concealing. Her husband's friend Richards was there, too, near her. It was he who had been in the newspaper office when intelligence of the railroad disaster was received, with Brently Mallard's name leading the list of killed. He had only taken the time to assure himself of its truth by a second telegram, and had hastened to forestall any less careful, less tender friend in bearing the sad message. She did not hear the story as many women have heard the same, with a paralyzed inability to accept its significance. She wept at once, with sudden, wild abandonment, in her sister's arms. When the storm of grief had spent itself she went away to her room alone. She would have no one follow her. There stood, facing the open window, a comfortable, roomy armchair. Into this she sank, pressed down by a physical exhaustion that haunted her body and seemed to reach into her soul. She could see in the open square before her house the tops of trees that were all aquiver with the new spring life. The delicious breath of rain was in the air. In the street below a peddler was crying his wares. The notes of a distant song which some one was singing reached her faintly, and countless sparrows were twittering in the eaves. There were patches of blue sky showing here and there through the clouds that had met and piled one above the other in the west facing her window. She was young, with a fair, calm face, whose lines bespoke repression and even a certain strength. But now there was a dull stare in her eyes, whose gaze was fixed away off yonder on one of those patches of blue sky. It was not a glance of reflection, but rather indicated a suspension of intelligent thought. There was something coming to her and she was waiting for it, fearfully. What was it? She did not know; it was too subtle and elusive to name. But she felt it, creeping out of the sky, reaching toward her through the sounds, the scents, the color that filled the air. Now her bosom rose and fell tumultuously. She was beginning to recognize this thing that was approaching to possess her, and she was striving to beat it back with her will - as powerless as her two white slender hands would have been."
test_list = test_text.split(".")

In [ ]:
len(test_list)

25

In [ ]:
def my_tokenizer(s):
  s = s.lower()
  tokens = nltk.tokenize.word_tokenize(s)
  tokens = [x for x in tokens if len(x) > 2]
  tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
  tokens = [t for t in tokens if t not in stopwords]
  return tokens

def token_to_vector(tokens):
  x = np.zeros(len(word_map))
  for t in tokens:
    if t in word_map:
      i = word_map[t]
      x[i] +=1
  x = x / x.sum()
  return x

counter = 0
for line in test_list:
  tokens = my_tokenizer(line)
  test_list[counter] = token_to_vector(tokens)
  counter+=1

test = np.array(test_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


In [ ]:
pred = model.predict(test[0:24])
pred

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished


array([0., 0., 0., 0., 3., 0., 0., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.,
       4., 0., 0., 0., 0., 0., 0.])

In [ ]:
chord_list = []
for x in pred:
  if x == 0:
    chord_list.append("d3")
  elif x==1:
    chord_list.append("b5")
  elif x==2:
    chord_list.append("a4")
  elif x==3:
    chord_list.append("g4")
  elif x==4:
    chord_list.append("c7")
  else:
    chord_list.append("f7")
print(chord_list)

['d3', 'd3', 'd3', 'd3', 'g4', 'd3', 'd3', 'd3', 'd3', 'd3', 'd3', 'd3', 'a4', 'd3', 'd3', 'd3', 'd3', 'c7', 'd3', 'd3', 'd3', 'd3', 'd3', 'd3']


### Testing 2

In [ ]:
test_text = "All-day is not the same, you need to wait and be patient for the good things to happen.\
Crying alone does not show that you are weak but it shows that you are strong.\
Be healthy and take care of yourself, but be happy with the beautiful things that make you, you.\
You can’t stop the waves, but you can learn to swim.\
I certainly had no idea Tom was angry. She thinks she is some kind of vicious animal. This was not a mangy or vicious creature standing before him, but a breathtakingly beautiful, gigantic wolf.\
There was fear in her eyes.Never let to know the reason of your sadness to others, they won’t get it.\
No medicine cures what happiness cannot. People are fearful of rising crime in the area."

test_list = test_text.split(".")
len(test_list)

12

In [ ]:
def my_tokenizer(s):
  s = s.lower()
  tokens = nltk.tokenize.word_tokenize(s)
  tokens = [x for x in tokens if len(x) > 2]
  tokens = [wordnet_lemmatizer.lemmatize(t) for t in tokens]
  tokens = [t for t in tokens if t not in stopwords]
  return tokens

def token_to_vector(tokens):
  x = np.zeros(len(word_map))
  for t in tokens:
    if t in word_map:
      i = word_map[t]
      x[i] +=1
  x = (x / x.sum())
  return x

counter = 0
for line in test_list:
  tokens = my_tokenizer(line)
  test_list[counter] = token_to_vector(tokens)
  counter+=1

test = np.array(test_list)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: RuntimeWarning: invalid value encountered in true_divide
  from ipykernel import kernelapp as app


In [ ]:
test = pd.DataFrame(test)
test = test.fillna(0)
pred = model.predict(test.values)
pred

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    0.1s finished


array([0., 0., 2., 0., 0., 0., 0., 0., 0., 0., 1., 0.])

In [ ]:
chord_list = []
for x in pred:
  if x == 0:
    chord_list.append("d3")
  elif x==1:
    chord_list.append("b5")
  elif x==2:
    chord_list.append("a4")
  elif x==3:
    chord_list.append("g4")
  elif x==4:
    chord_list.append("c7")
  else:
    chord_list.append("f7")
print(chord_list)

['d3', 'd3', 'a4', 'd3', 'd3', 'd3', 'd3', 'd3', 'd3', 'd3', 'b5', 'd3']
